In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
from pprint import pprint

# Resource File (CSV)
five_cities = pd.read_csv("output_data/restaurants_category_10_five_cities.csv")
five_cities.head()

,Unnamed: 0,Restaurant_id,Name,Locality,Address,City,Latitude,Longitude,Zip Code,Cuisines,Price Range,Average Cost for two,User Rating,Rating Text,Votes,all_reviews_count
0,0,15547004,Restaurant Hubert,CBD,"15 Bligh Street, CBD, Sydney",Sydney,-33.865348,151.210624,2000.0,"French, European",4,150,4.9,Excellent,589,224
1,1,16558798,Quay,Circular Quay,"Upper Level, Overseas Passenger Terminal 5 Hic...",Sydney,-33.858029,151.209970,2000.0,Modern Australian,4,500,4.9,Excellent,1366,454
2,2,16559171,Tetsuya's,CBD,"529 Kent Street, CBD, Sydney",Sydney,-33.875143,151.204932,NaN,Japanese,4,440,4.9,Excellent,1234,329
3,3,16569454,LuMi Bar & Dining,Pyrmont,"56 Pirrama Road, \tPyrmont, Pyrmont, Sydney",Sydney,-33.867137,151.197517,2009.0,"Italian, Japanese",4,190,4.9,Excellent,452,196
4,4,15545439,Manpuku,Chatswood,"226 Victoria Avenue, Chatswood, Sydney",Sydney,-33.794417,151.189542,2067.0,"Japanese, Ramen",2,40,4.9,Excellent,486,190


In [2]:
#Reorganising columns so Cuisine is at the end of the dataframe - dropping unnamed.
new_df = five_cities[['Restaurant_id',"Name", "Locality", "Address", "City", "Latitude", "Longitude", "Zip Code", "Price Range", 
                      "Average Cost for two", "User Rating", "Rating Text", "Votes", "Cuisines"]]
new_df

,Restaurant_id,Name,Locality,Address,City,Latitude,Longitude,Zip Code,Price Range,Average Cost for two,User Rating,Rating Text,Votes,Cuisines
0,15547004,Restaurant Hubert,CBD,"15 Bligh Street, CBD, Sydney",Sydney,-33.865348,151.210624,2000.0,4,150,4.9,Excellent,589,"French, European"
1,16558798,Quay,Circular Quay,"Upper Level, Overseas Passenger Terminal 5 Hic...",Sydney,-33.858029,151.209970,2000.0,4,500,4.9,Excellent,1366,Modern Australian
2,16559171,Tetsuya's,CBD,"529 Kent Street, CBD, Sydney",Sydney,-33.875143,151.204932,NaN,4,440,4.9,Excellent,1234,Japanese
3,16569454,LuMi Bar & Dining,Pyrmont,"56 Pirrama Road, \tPyrmont, Pyrmont, Sydney",Sydney,-33.867137,151.197517,2009.0,4,190,4.9,Excellent,452,"Italian, Japanese"
4,15545439,Manpuku,Chatswood,"226 Victoria Avenue, Chatswood, Sydney",Sydney,-33.794417,151.189542,2067.0,2,40,4.9,Excellent,486,"Japanese, Ramen"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,16586992,Jolley's Boathouse,City Centre,"1 Jolleys Lane, City Centre, Adelaide",Adelaide,-34.917011,138.600296,5000.0,4,120,4.2,Very Good,233,"Seafood, Modern Australian"
496,16588907,The Gallery,"Waymouth Street, City Centre","30 Waymouth Street, Adelaide, City Centre, Ade...",Adelaide,-34.925872,138.598311,5000.0,4,100,4.2,Very Good,166,"Tapas, Bar Food"
497,16587014,Lenzerheide Restaurant,Kingswood,"146 Belair Road, Kingswood, Adelaide",Adelaide,-34.972047,138.609192,5062.0,4,120,4.2,Very Good,233,European
498,16589555,Sukhumvit Soi 38,City Centre,"54 Pulteney Street, Adelaide",Adelaide,-34.923131,138.605479,5000.0,3,90,4.2,Very Good,145,Thai


In [4]:
#Separating Cuisines data by comma and placing the data into new columns
df = pd.concat([new_df, new_df['Cuisines'].str.split(', ', expand=True)], axis=1)
df

,Restaurant_id,Name,Locality,Address,City,Latitude,Longitude,Zip Code,Price Range,Average Cost for two,...,Votes,Cuisines,0,1,2,3,4,5,6,7
0,15547004,Restaurant Hubert,CBD,"15 Bligh Street, CBD, Sydney",Sydney,-33.865348,151.210624,2000.0,4,150,...,589,"French, European",French,European,None,None,None,None,None,None
1,16558798,Quay,Circular Quay,"Upper Level, Overseas Passenger Terminal 5 Hic...",Sydney,-33.858029,151.209970,2000.0,4,500,...,1366,Modern Australian,Modern Australian,None,None,None,None,None,None,None
2,16559171,Tetsuya's,CBD,"529 Kent Street, CBD, Sydney",Sydney,-33.875143,151.204932,NaN,4,440,...,1234,Japanese,Japanese,None,None,None,None,None,None,None
3,16569454,LuMi Bar & Dining,Pyrmont,"56 Pirrama Road, \tPyrmont, Pyrmont, Sydney",Sydney,-33.867137,151.197517,2009.0,4,190,...,452,"Italian, Japanese",Italian,Japanese,None,None,None,None,None,None
4,15545439,Manpuku,Chatswood,"226 Victoria Avenue, Chatswood, Sydney",Sydney,-33.794417,151.189542,2067.0,2,40,...,486,"Japanese, Ramen",Japanese,Ramen,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,16586992,Jolley's Boathouse,City Centre,"1 Jolleys Lane, City Centre, Adelaide",Adelaide,-34.917011,138.600296,5000.0,4,120,...,233,"Seafood, Modern Australian",Seafood,Modern Australian,None,None,None,None,None,None
496,16588907,The Gallery,"Waymouth Street, City Centre","30 Waymouth Street, Adelaide, City Centre, Ade...",Adelaide,-34.925872,138.598311,5000.0,4,100,...,166,"Tapas, Bar Food",Tapas,Bar Food,None,None,None,None,None,None
497,16587014,Lenzerheide Restaurant,Kingswood,"146 Belair Road, Kingswood, Adelaide",Adelaide,-34.972047,138.609192,5062.0,4,120,...,233,European,European,None,None,None,None,None,None,None
498,16589555,Sukhumvit Soi 38,City Centre,"54 Pulteney Street, Adelaide",Adelaide,-34.923131,138.605479,5000.0,3,90,...,145,Thai,Thai,None,None,None,None,None,None,None
